# ROS2 Documentation for Beginners
## Chapter 3 Supplement of the ROS2 Communication Mechanism

Chapter 2 introduces the core content of the ROS 2 communication mechanism, focusing more on a high-level overview of the framework. This chapter provides supplementary details about the communication mechanism, such as building a distributed framework, handling naming conflicts, commonly used APIs, and tools related to the communication mechanism. These supplementary topics are somewhat scattered but not complex. Additionally, at the end of this chapter, several exercises are included to reinforce the understanding of the ROS 2 communication mechanism.communication mechanism in ROS 2.


### Overview of this chapter ### 

| Chapter | Learning Content | Learning Outcome |
| ------- | ---------------- | ---------------- |
| 3.1 Distributed | Concept of distributed communication, application scenarios, and communication rules. | Able to independently set up a distributed communication framework. |
| 3.2 Workspace Overlay | Issues that may arise when package names conflict across different workspaces. | Strive to avoid package name conflicts in the future. |
| 3.3 Meta Packages | Concept, application scenarios, and creation rules for meta packages. | Able to customize meta packages. |
| 3.4 Node Naming Conflicts | Issues and solutions for node naming conflicts. | Able to resolve node naming conflicts appropriately. |
| 3.5 Topic Naming Conflicts | Issues and solutions for topic naming conflicts. | Able to resolve topic naming conflicts appropriately. |
| 3.6 Time-Related APIs | Time-related APIs, such as `Rate`, `Time`, and `Duration`. | Able to use time-related APIs. |
| 3.7 Communication Tools | Common command-line tools and the `rqt` toolbox for communication. | Able to use communication tools to improve program development and debugging efficiency. |
| 3.8 Communication Mechanism Practice | Exercises related to various communication mechanisms. | Reinforces understanding of the ROS 2 communication mechanism. |
| 3.9 Chapter Summary | Summary of key points. | Review of key concepts. |



## 3.1 Data Distribution Se Communication


In ROS 2, although there are various communication methods, the components of each method are similar. For example: communication involves two or more parties, different communication objects need to be linked, each has its own model, and data exchange always requires a data carrier. This section will introduce some of the terminology involved in ROS 2 communication.


**Scenario**
In many robotics-related application scenarios, multiple ROS 2 devices work together, such as in autonomous vehicle fleets, drone swarms, and remote control operations. But how do different ROS 2 devices communicate with each other?

**Concept**
Distributed communication refers to a communication strategy that enables data exchange across different hosts over a network.

ROS 2 itself is a distributed communication framework, making it easy to achieve communication between different devices. ROS 2 is based on DDS (Data Distribution Service), and when devices are on the same network, distributed communication can be achieved through DDS's Domain ID mechanism (ROS_DOMAIN_ID). The basic process is as follows: before launching a node, you can set the Domain ID value. Nodes with the same Domain ID can discover and communicate with each other, while nodes with different Domain IDs cannot. By default, all nodes use Domain ID 0 when launched, meaning that as long as devices are on the same network, no additional configuration is needed for nodes on different ROS 2 devices to achieve distributed communication.

**Application**
Distributed communication is widely applicable. As mentioned above, in robotic swarms, robots may need to obtain information about the speed, position, and trajectory of neighboring robots. In remote control scenarios, the control unit may need to access environmental information collected by the robot and send control commands. All of these data exchanges rely on distributed communication.

**Implementation**
For multi-machine communication, you can group nodes by Domain ID. Nodes within the same group can communicate freely, while nodes in different groups cannot. If all nodes belong to the same group, the default Domain ID can be used. To divide nodes into multiple groups, you can set the Domain ID of a node before launching it in the terminal (e.g., set it to 6) using the following command:

```bash
export ROS_DOMAIN_ID=6
```

After executing this command, the node will be assigned to the domain with ID 6.

To set a uniform Domain ID for all nodes on the current device, use the following command:

```bash

echo "export ROS_DOMAIN_ID=6" >> ~/.bashrc
```
After completing this command, restart the terminal, and all nodes that are launched will automatically be assigned to the domain with ID 6.


**Note**
There are certain constraints when setting the value of `ROS_DOMAIN_ID`:

- It is recommended to set `ROS_DOMAIN_ID` within the range [0, 101], inclusive of 0 and 101.
- The total number of nodes within each Domain ID is limited to 120 or fewer.
- If the Domain ID is set to 101, the total number of nodes in that domain must be 54 or fewer.

**DDS Domain ID Calculation Rules**

The calculation rules for the Domain ID values in DDS are as follows:

- DDS communication is based on the TCP/IP or UDP/IP network protocols, which require specifying a port number. The port number is an unsigned 2-byte integer with a range of [0, 65535].
- Port allocation follows specific rules and cannot be arbitrarily assigned. According to the DDS protocol, 7400 is the starting port, making the usable port range [7400, 65535]. Additionally, by default, each Domain ID uses 250 ports, so the total number of possible Domain IDs is `(65535 - 7400) / 250 = 232`, meaning the Domain ID range is [0, 231].
- Operating systems also reserve certain ports. When using ports in DDS, these reserved ports should be avoided to prevent conflicts. Different operating systems have different reserved ports. As a result, on Linux, the usable Domain IDs are [0, 101] and [215, 231], while on Windows and macOS, the usable Domain IDs are [0, 166]. Therefore, for cross-platform compatibility, it is recommended to set Domain IDs within the range [0, 101].
- Each Domain ID defaults to using 250 ports, and each ROS 2 node occupies two ports. According to the DDS protocol, within each Domain ID's port range, the first and second ports are the Discovery Multicast and User Multicast ports. The Discovery Unicast and User Unicast ports for the first node in the domain start at the 11th and 12th ports, respectively. Ports for subsequent nodes are assigned sequentially. Thus, the maximum number of nodes per Domain ID is `(250 - 10) / 2 = 120`.
- Special Case: When the Domain ID is set to 101, the second half of the port range falls within the operating system’s reserved ports, limiting the maximum number of nodes to 54.

These calculation rules are primarily for reference.

```
Domain ID:      0
Participant ID: 0

Discovery Multicast Port: 7400
User Multicast Port:      7401
Discovery Unicast Port:   7410
User Unicast Port:        7411

---

Domain ID:      1
Participant ID: 2
Discovery Multicast Port: 7650
User Multicast Port:      7651
Discovery Unicast Port:   7664
User Unicast Port:        7665
```

## 3.2 MetaPackage 
**Scenario**

Completing a system-level feature may involve multiple packages. For example, implementing a robot navigation module could include different sub-packages for mapping, localization, path planning, etc. When a user installs this module, do they need to install each package individually?

Clearly, installing packages one by one is inefficient. In ROS 2, there is a way to bundle different packages into a single package. When installing a module, users can simply install the bundled package, known as a metapackage.


**Concepts**

A **Metapackage** is a concept from Linux file management systems. In ROS 2, it is a "virtual package" that contains no actual content but depends on other packages. This allows packages to be grouped together, functioning like an index in a book, listing the sub-packages included and where to download them.

For example:

When you use the command `sudo apt install ros-<ros2-distro>-desktop` to install ROS 2, a metapackage is used. This metapackage depends on other packages in ROS 2, so installing it will also install its dependencies.


**Purpose**

Metapackages make installation convenient for users. With a single package, all related packages are grouped together and installed in one go.

**Implementation**
- create a package. 
  ``` ros2 pkg create tutorials_plumbing ```
- Edit the package.xml
  ```xml
  <?xml version="1.0"?>
  <?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
  <package format="3">
      <name>tutorails_plumbing</name>
      <version>0.0.0</version>
      <description>TODO: Package description</description>
      <maintainer email="ros2@todo.todo">ros2</maintainer>
      <license>TODO: License declaration</license>

      <buildtool_depend>ament_cmake</buildtool_depend>

      <exec_depend>base_interfaces_demo</exec_depend>
      <exec_depend>cpp01_topic</exec_depend>
      <exec_depend>cpp02_service</exec_depend>
      <exec_depend>cpp03_action</exec_depend>
      <exec_depend>cpp04_param</exec_depend>
      <exec_depend>py01_topic</exec_depend>
      <exec_depend>py02_service</exec_depend>
      <exec_depend>py03_action</exec_depend>
      <exec_depend>py04_param</exec_depend>


      <export>
        <build_type>ament_cmake</build_type>
      </export>
    </package>
```

- Edit the CMakeLists.txt
  ```cmake
      cmake_minimum_required(VERSION 3.8)
      project(tutorails_plumbing)

      if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
        add_compile_options(-Wall -Wextra -Wpedantic)
      endif()

      find_package(ament_cmake REQUIRED)

      ament_package()
```

### 3.3 Time related APIs

In previous exmaples, we used the `Rate` class to control the frequency of message publishing. In this section, we will introduce other time-related APIs in ROS 2.

<img src="figs/excution_demo.gif" alt="Execution Demo" width="300" autoplay>


#### 3.3.1 Rate Class ####

In the topic communication example in Chapter 2, the topic publisher is required to publish messages at a certain frequency. In our implementation, we use a timer to control the publishing rate. However, besides timers, ROS 2 also provides the `Rate` class, which can also be used to control the execution frequency of a program.

**Rate Class in rclpy**
Example: Display a text periodically at a fixed frequency.

```python

import rclpy
import threading
from rclpy.timer import Rate

rate = None
node = None

def do_some():
    global rate
    global node
    while rclpy.ok():
        node.get_logger().info("hello ---------")
        # Sleep
        rate.sleep()

def main():
    global rate
    global node
    rclpy.init()    
    node = rclpy.create_node("rate_demo")
    # Create Rate Object
    rate = node.create_rate(1.0)

    # Create sub-thread
    thread = threading.Thread(target=do_some)
    thread.start()

    rclpy.shutdown()

if __name__ == "__main__":
    main()

```
#### 3.3.2 Time ####
Example: Create Time function.

```python
import rclpy
from rclpy.time import Time
def main():
    rclpy.init()
    node = rclpy.create_node("time_demo")
    # Create Time Object
    right_now = node.get_clock().now()
    t1 = Time(seconds=10,nanoseconds=500000000)

    node.get_logger().info("s = %.2f, ns = %d" % (right_now.seconds_nanoseconds()[0], right_now.seconds_nanoseconds()[1]))
    node.get_logger().info("s = %.2f, ns = %d" % (t1.seconds_nanoseconds()[0], t1.seconds_nanoseconds()[1]))
    node.get_logger().info("ns = %d" % right_now.nanoseconds)
    node.get_logger().info("ns = %d" % t1.nanoseconds)
    rclpy.shutdown()

if __name__ == "__main__":
    main()

```
#### 3.3.3 Duration Class ####
Create a Duration object.

```python

import rclpy
from rclpy.duration import Duration

def main():
    rclpy.init()    

    node = rclpy.create_node("duration_demo")
    du1 = Duration(seconds = 2,nanoseconds = 500000000)
    node.get_logger().info("ns = %d" % du1.nanoseconds)

    rclpy.shutdown()

if __name__ == "__main__":

    main()

```
```python
import rclpy
from rclpy.time import Time
from rclpy.duration import Duration

def main():
    rclpy.init()
    node = rclpy.create_node("time_opt_node")
    t1 = Time(seconds=10)
    t2 = Time(seconds=4)

    du1 = Duration(seconds=3)
    du2 = Duration(seconds=5)

    # Compare
    node.get_logger().info("t1 >= t2 ? %d" % (t1 >= t2))
    node.get_logger().info("t1 < t2 ? %d" % (t1 < t2))
    # Compute
    t3 = t1 + du1
    t4 = t1 - t2    
    t5 = t1 - du1

    node.get_logger().info("t3 = %d" % t3.nanoseconds)
    node.get_logger().info("t4 = %d" % t4.nanoseconds)
    node.get_logger().info("t5 = %d" % t5.nanoseconds)

    # Compare
    node.get_logger().info("-" * 80)
    node.get_logger().info("du1 >= du2 ? %d" % (du1 >= du2))
    node.get_logger().info("du1 < du2 ? %d" % (du1 < du2))

    rclpy.shutdown()

if __name__ == "__main__":
    main()
```


### 3.3 Communication Tools ###

**Scenario**  
In the previous chapter, we explored various communication mechanisms in ROS 2, understanding the implementation process, related APIs, and characteristics of each communication model. Now, we will discuss some practical issues that may arise during development:

1. After launching a complete robot system, the setup can be quite complex, possibly consisting of dozens or even hundreds of nodes. Each node might contain one or more communication entities (such as topic publishers, topic subscribers, servers, clients, action servers, action clients, parameter servers, and parameter clients). Additionally, various message (`msg`), service (`srv`), or action interfaces are used during communication. So, how can developers easily access information about these nodes, topics, services, actions, parameters, and interfaces?
2. When implementing communication, at least two parties are involved. Once one side is completed, how can we verify if the program functions correctly?
3. During topic communication, a publishing frequency is set in the publisher’s code. How can we verify if the actual publishing frequency matches the specified frequency during runtime?
   
ROS 2 provides tools that efficiently address these questions. This section introduces these tools and how to use them.

**Concept**  
In ROS 2, tools related to communication mechanisms fall into two categories: command-line tools and graphical tools (rqt). The command-line tools are a collection of terminal commands, while rqt is a set of visualization tools developed by ROS 2 based on the QT framework, specifically for robot development.

**Purpose**  
These tools facilitate program debugging, improve development efficiency, and enhance the user experience.


#### 3.3.1 Command-Line Tools ####

Comment Commands in ROS2:
- *ros2 node*: Manage nodes.
- *ros2 interface*: Display message, service, and action interfaces.
- *ros2 topic*: Manage topics.
- *ros2 service*: Manage services.
- *ros2 action*: Manage actions.
- *ros2 param*: Manage parameters.
  
You can check the documentation like ```bash ros2 node -h``` to get more information.

1. **ros2 node**  
   - List all nodes: `ros2 node list`
   - Get information about a node: `ros2 node info /node_name`
   - Get information about all nodes: `ros2 node info -a`
   - Get information about a node's topic, service, and action interfaces: `ros2 node info /node_name --sub --pub --srv --act`
   - Get information about all nodes' topic, service, and action interfaces: `ros2 node info -a --sub --pub --srv --act`
   - Get information about a node's parameters: `ros2 node info /node_name --params`
   - Get information about all nodes' parameters: `ros2 node info -a --params`
2. **ros2 interface**  
   - Display message, service, and action interfaces: `ros2 interface show package_name/msg_name`
   - Display all message, service, and action interfaces: `ros2 interface list`
   - Display all message, service, and action interfaces in a package: `ros2 interface list package_name`
   - packages: `ros2 interface packages`
   - proto: `ros2 interface proto package_name/msg_name`
   - show: `ros2 interface show package_name/msg_name`
3. **ros2 topic**
   - bw: `ros2 topic bw /topic_name`        # Display the bandwidth of the topic
   - delay: `ros2 topic delay /topic_name` # Display the delay of the topic
   - echo: `ros2 topic echo /topic_name` # Display the message content of the topic
   - find: `ros2 topic find /topic_name` # Find the node that publishes the topic
   - hz: `ros2 topic hz /topic_name` # Display the publishing frequency of the topic
   - info: `ros2 topic info /topic_name`# Display information about the topic
   - list: `ros2 topic list` # List all topics
   - pub: `ros2 topic pub /topic_name package_name/msg_name` # Publish a message to the topic
   - type: `ros2 topic type /topic_name` # Display the message type of the topic 
4. **ros2 service**
   - find: `ros2 service find /service_name` # Find the node that provides the service
   - list: `ros2 service list` # List all services
   - type: `ros2 service type /service_name` # Display the service type
   - call: `ros2 service call /service_name package_name/srv_name` # Call the service
5. **ros2 action**
   - list: `ros2 action list` # List all actions
   - send_goal: `ros2 action send_goal /action_name package_name/action_name` # Send a goal to the action
   - info: `ros2 action info /action_name` # Display information about the action
6. **ros2 param**
   - get: `ros2 param get /node_name param_name` # Get the parameter value
   - list: `ros2 param list /node_name` # List all parameters
   - set: `ros2 param set /node_name param_name value` # Set the parameter value
   - describe: `ros2 param describe /node_name param_name` # Describe the parameter
   - delete: `ros2 param delete /node_name param_name` # Delete the parameter
   - describe: `ros2 param describe /node_name param_name` # Describe the parameter
   - dump: `ros2 param dump /node_name` # Dump all parameters
   - load: `ros2 param load /node_name file_name` # Load parameters from a file

### 3.3.2 rqt Toolbox ###

**Installation**
```bash
sudo apt install ros-humble-rqt*
```

**Execution**
```bash
rqt
```
```bash
ros2 run rqt_topic rqt_topic
```

![RQT Toolbox](figs/3.7.2RQT.gif)



1. topic plugin
   
   ![RQT Toolbox](figs/3.7.2RQTtooltopic.gif)

2. service plugin
   
   ![RQT Toolbox](figs/3.7.2RQTtoolservice.gif)

3. para plugin
   
   ![RQT Toolbox](figs/3.7.2RQTtoolpara.gif)


